# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column
from desiutil.names import radec_to_desiname
from desiutil.log import get_logger, DEBUG
from desispec.io.photo import gather_tractorphot, gather_targetphot
from desitarget.targets import decode_targetid
import specprodDB.load as db
specprod = os.environ['SPECPROD']
tileid = 3867
survey, program = 'main', 'dark'

## Initialize Database

In [2]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema='daily', hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=True)

INFO:load.py:1415:setup_db: Begin creating tables.
INFO:load.py:1418:setup_db: Finished creating tables.


## Read tiles file

In [3]:
specprod_dir = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)
tiles_file = os.path.join(specprod_dir, f'tiles-{specprod}.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
tiles_table

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
70004,unknown,--,unknown,unknown,4,3600.0,116.0,20.7,0.0,3619.8,0.0,1000.0,obsend,3470.8,3619.8,3784.0,3056.6,unknown,0.9,20200219
70508,unknown,--,unknown,unknown,6,1800.0,133.4125,11.6818,0.0,108.5,0.0,1000.0,obsstart,93.6,108.5,95.7,50.6,unknown,0.9,20200225
70506,unknown,--,unknown,unknown,3,900.0,133.4125,11.6818,0.0,61.6,0.0,1000.0,obsstart,53.6,61.6,54.9,33.7,unknown,0.9,20200225
70512,unknown,--,unknown,unknown,7,3150.0,132.85,12.32,0.0,389.8,0.0,1000.0,obsstart,351.9,389.8,364.7,227.2,unknown,0.9,20200226
70514,unknown,--,unknown,unknown,16,1680.0,132.85,12.32,0.0,118.3,0.0,1000.0,obsstart,112.0,118.3,120.2,74.4,unknown,0.9,20200227
70502,unknown,--,unknown,unknown,41,8415.0,180.0,-0.5,0.0,358.2,0.0,1000.0,obsstart,317.9,358.2,325.0,260.2,unknown,0.9,20200227
70513,unknown,--,unknown,unknown,13,2220.0,133.42,11.65,0.0,3.7,0.0,1000.0,obsstart,3.4,3.7,3.5,2.5,unknown,0.9,20200229
70500,unknown,--,unknown,unknown,14,7000.0,119.0,50.0,0.0,1389.0,0.0,1000.0,obsend,1145.4,1389.0,1160.5,770.4,unknown,0.9,20200303
70005,unknown,--,unknown,unknown,17,13500.0,158.0,25.0,0.0,8463.1,0.0,1000.0,obsend,8088.5,8463.1,8858.6,9174.7,unknown,0.9,20200303


In [5]:
tiles_table[tiles_table['TILEID'] == tileid]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
3867,main,--,dark,maindark,1,1355.5,130.832,69.075,1008.4,1108.3,966.7,1000.0,obsend,1108.3,1191.1,1081.0,1111.7,dark,0.85,20211126


In [15]:
tiles_table[tiles_table['TILEID'] == tileid]['PROGRAM'].mask

array([ True])

In [24]:
[(c.name, c.type) for c in db.Tile.__table__.columns]

[('tileid', Integer()),
 ('survey', String(length=20)),
 ('program', String(length=6)),
 ('faprgrm', String(length=20)),
 ('faflavor', String(length=20)),
 ('nexp', BigInteger()),
 ('exptime', DOUBLE_PRECISION()),
 ('tilera', DOUBLE_PRECISION()),
 ('tiledec', DOUBLE_PRECISION()),
 ('efftime_etc', DOUBLE_PRECISION()),
 ('efftime_spec', DOUBLE_PRECISION()),
 ('efftime_gfa', DOUBLE_PRECISION()),
 ('goaltime', DOUBLE_PRECISION()),
 ('obsstatus', String(length=20)),
 ('lrg_efftime_dark', DOUBLE_PRECISION()),
 ('elg_efftime_dark', DOUBLE_PRECISION()),
 ('bgs_efftime_bright', DOUBLE_PRECISION()),
 ('lya_efftime_dark', DOUBLE_PRECISION()),
 ('goaltype', String(length=20)),
 ('mintfrac', DOUBLE_PRECISION()),
 ('lastnight', Integer())]

In [33]:
row_index = np.where(tiles_table['TILEID'] == tileid)[0]
assert len(row_index) == 1
load_tile = dict()
for c in db.Tile.__table__.columns:
    if c.name == 'program' and tiles_table[row_index[0]]['PROGRAM'].mask:
        load_tile[c.name] = 'dark'
    else:
        load_tile[c.name] = tiles_table[row_index[0]][c.name.upper()]
new_tile = db.Tile(**load_tile)

In [34]:
new_tile.tileid

3867

In [36]:
# db.dbSession.rollback()
db.dbSession.add(new_tile)
db.dbSession.commit()

/tmp/ipykernel_622972/1377210702.py:1: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  db.dbSession.rollback()


## Read exposures file

In [38]:
exposures_file = os.path.join(specprod_dir, f'exposures-{specprod}.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
exposures_table[exposures_table['TILEID'] == tileid]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20211126,110987,3867,130.832,69.075,59545.42359139,main,dark,dark,maindark,1355.54345703125,1108.266357421875,1000.0,dark,0.85,1.2937510013580322,0.0328538715839386,0.7816540002822876,1008.3909912109375,138.49758911132812,36.12108612060547,91.2107925415039,94.20612081214786,7721.27587890625,12680.3271484375,2221.425537109375,16633.6796875,1108.266357421875,1191.079345703125,1080.9786376953125,1111.71786024777,1063.00244140625,1162.42626953125,1178.088623046875,1.0194541169953857,0.784009214278641,0.774699396350431,0.5381829509439544,0.24051108697514237,1.2897602996518225,1.229704022020451,1.1984016686497583,1.2852353271869736,20.334894382343847,20.66803869771934,20.317365959665477,18.984880968726785,966.671222839215,966.671222839215,1050.159859268238,1518.3969558168317


In [69]:
from astropy.time import Time
from pytz import utc

row_index = np.where(exposures_table['TILEID'] == tileid)[0]
assert len(row_index) > 0
data_columns = list()
for c in db.Exposure.__table__.columns:
    if c.name == 'date_obs':
        data_columns.append(list(map(utc.localize, Time(exposures_table[row_index]['MJD'], format='mjd').to_value('datetime').tolist())))
    else:
        data_columns.append(exposures_table[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_exposures = [db.Exposure(**(dict([(c.name, d) for c, d in zip(db.Exposure.__table__.columns, row)]))) for row in data_rows]
load_exposures

[Exposure(night=20211126, expid=110987, tileid=3867)]

In [70]:
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

In [76]:
from specprodDB.util import cameraid
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    data_columns = list()
    for c in db.Frame.__table__.columns:
        if c.name == 'frameid':
            data_columns.append((100*frames_table[row_index]['EXPID'] + np.array([cameraid(c) for c in frames_table[row_index]['CAMERA']], dtype=frames_table[row_index]['EXPID'].dtype)).tolist())
        else:
            data_columns.append(frames_table[row_index][c.name.upper()].tolist())
    data_rows = list(zip(*data_columns))
    load_frames += [db.Frame(**(dict([(c.name, d) for c, d in zip(db.Frame.__table__.columns, row)]))) for row in data_rows]
load_frames

[Frame(expid=110987, camera='b0'),
 Frame(expid=110987, camera='b1'),
 Frame(expid=110987, camera='b2'),
 Frame(expid=110987, camera='b3'),
 Frame(expid=110987, camera='b4'),
 Frame(expid=110987, camera='b5'),
 Frame(expid=110987, camera='b6'),
 Frame(expid=110987, camera='b7'),
 Frame(expid=110987, camera='b8'),
 Frame(expid=110987, camera='b9'),
 Frame(expid=110987, camera='r0'),
 Frame(expid=110987, camera='r1'),
 Frame(expid=110987, camera='r2'),
 Frame(expid=110987, camera='r3'),
 Frame(expid=110987, camera='r4'),
 Frame(expid=110987, camera='r5'),
 Frame(expid=110987, camera='r6'),
 Frame(expid=110987, camera='r7'),
 Frame(expid=110987, camera='r8'),
 Frame(expid=110987, camera='r9'),
 Frame(expid=110987, camera='z0'),
 Frame(expid=110987, camera='z1'),
 Frame(expid=110987, camera='z2'),
 Frame(expid=110987, camera='z3'),
 Frame(expid=110987, camera='z4'),
 Frame(expid=110987, camera='z5'),
 Frame(expid=110987, camera='z6'),
 Frame(expid=110987, camera='z7'),
 Frame(expid=110987,

In [77]:
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [78]:
def no_sky(catalog):
    """Identify objects in `catalog` that are not sky targets.
    
    Parameters
    ----------
    catalog : :class:`~astropy.table.Table`
        Any Table containing a ``TARGETID`` column.
    
    Returns
    -------
    :class:`numpy.ndarray`
        The indexes of rows that are not sky targets.
    """
    _, _, _, _, sky, _ = decode_targetid(catalog['TARGETID'])
    return np.where((sky == 0) & (catalog['TARGETID'] > 0))[0]

In [79]:
fiberassign_file = os.path.join(os.environ['DESI_ROOT'], 'target', 'fiberassign', 'tiles', 'trunk', f'{tileid:06d}'[0:3], f'fiberassign-{tileid:06d}.fits.gz')
potential_targets_table = Table.read(fiberassign_file, format='fits', hdu='TARGETS')
no_sky_rows = no_sky(potential_targets_table)
potential_cat = Table()
potential_cat['TARGETID'] = potential_targets_table['TARGETID'][no_sky_rows]
potential_cat['TILEID'] = tileid
potential_cat['TARGET_RA'] = potential_targets_table['RA'][no_sky_rows]
potential_cat['TARGET_DEC'] = potential_targets_table['DEC'][no_sky_rows]
# potential_cat['PETAL_LOC'] = potential_targets_table['PETAL_LOC'][no_sky_rows]
potential_cat['SURVEY'] = new_tile.survey
potential_cat['PROGRAM'] = 'dark'
potential_cat

TARGETID,TILEID,TARGET_RA,TARGET_DEC,SURVEY,PROGRAM
int64,int64,float64,float64,str4,str4
2404005925879810,3867,133.4399101445135,67.82437322108153,main,dark
2404005925879811,3867,133.44185934546013,67.82823149466229,main,dark
2404005925879812,3867,133.27805902114852,67.83948363786199,main,dark
2404005925879813,3867,133.28847089528776,67.84041846265472,main,dark
2404008190803968,3867,128.18399465466373,68.06496622501886,main,dark
2404008190803969,3867,128.19164716436438,68.06950585763119,main,dark
2404008199192577,3867,129.52376427694398,68.06131962761214,main,dark
2404008199192578,3867,129.5248554721843,68.05751118313499,main,dark
2404008207581184,3867,130.90646227549064,67.98937459790655,main,dark


In [80]:
potential_targetphot = gather_targetphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
potential_targetphot['SURVEY'] = potential_cat['SURVEY']
potential_targetphot['PROGRAM'] = potential_cat['PROGRAM']
potential_targetphot['TILEID'] = potential_cat['TILEID']
potential_targetphot['DESINAME'] = radec_to_desiname(potential_targetphot['RA'], potential_targetphot['DEC'])
inan = np.logical_or(np.isnan(potential_targetphot['PMRA']), np.isnan(potential_targetphot['PMDEC']))
if np.any(inan):
    potential_targetphot['PMRA'][inan] = 0.0
    potential_targetphot['PMDEC'][inan] = 0.0
potential_targetphot

RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1,LC_FLUX_W2,LC_FLUX_IVAR_W1,LC_FLUX_IVAR_W2,LC_NOBS_W1,LC_NOBS_W2,LC_MJD_W1,LC_MJD_W2,SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SCND_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SURVEY,PROGRAM,TILEID,DESINAME
int16,int32,str8,int32,str4,float64,float32,float64,float32,float32[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32[15],float32[15],float32[15],float32[15],int16[15],int16[15],float64[15],float64[15],float32,float32,float32,float32,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,uint8,float32,float32,float32,float32,float32,float32,str1,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,str4,str4,int64,str22
0,0,,0,,133.4399101445135,0.0,67.82437322108153,0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015.5,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,,2404005925879810,0.1729051562495022,1,1700,5,0,0,4611686018427387904,0,0,0,0,0,0,0,0,0,0,0,70368744177664,0,0,0,main,dark,3867,DESI J133.4399+67.8243
0,0,,0,,133.44185934546013,0.0,67.82823149466229,0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015.5,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,,2404005925879811,0.49123348190990845,1,1700,5,0,0,4611686018427387904,0,0,0,0,0,0,0,0,0,0,0,70368744177664,0,0,0,main,dark,3867,DESI J133.4418+67.8282
0,0,,0,,133.27805902114852,0.0,67.83948363786199,0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015.5,0,0,0,

In [81]:
potential_tractorphot = gather_tractorphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [85]:
assert (np.where(potential_tractorphot['RELEASE'] == 0)[0] == np.where(potential_tractorphot['BRICKNAME'] == '')[0]).all()

In [87]:
row_index = np.where(potential_tractorphot['BRICKNAME'] != '')[0]
expand_dchisq = ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Photometry.__table__.columns:
        if c.name == 'brick_objid':
            data_columns.append(potential_tractorphot[row_index]['OBJID'].tolist())
        elif c.name == 'morphtype':
            data_columns.append(potential_tractorphot[row_index]['TYPE'].tolist())
        elif c.name in expand_dchisq:
            j = expand_dchisq.index(c.name)
            data_columns.append(potential_tractorphot[row_index]['DCHISQ'][:, j].tolist())
        else:
            data_columns.append(potential_tractorphot[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_photometry = [db.Photometry(**(dict([(c.name, d) for c, d in zip(db.Photometry.__table__.columns, row)]))) for row in data_rows]
load_photometry[:20]

[Photometry(targetid=2404014939439105),
 Photometry(targetid=2707468807307264),
 Photometry(targetid=2707468807307265),
 Photometry(targetid=2707471109980160),
 Photometry(targetid=2707471109980161),
 Photometry(targetid=2707471118368768),
 Photometry(targetid=2707471122563072),
 Photometry(targetid=2707473400070144),
 Photometry(targetid=2707473404264448),
 Photometry(targetid=2707475673382912),
 Photometry(targetid=2707475677577216),
 Photometry(targetid=2707475677577217),
 Photometry(targetid=2707475690160128),
 Photometry(targetid=2707475690160129),
 Photometry(targetid=2707475706937344),
 Photometry(targetid=2707475706937345),
 Photometry(targetid=2707477934112768),
 Photometry(targetid=2707477938307072),
 Photometry(targetid=2707477955084288),
 Photometry(targetid=2707477959278592),
 Photometry(targetid=2707477959278593),
 Photometry(targetid=2707480136122368),
 Photometry(targetid=2707480157093888),
 Photometry(targetid=2707480157093889),
 Photometry(targetid=2707480161288192),


In [88]:
# db.dbSession.rollback()
db.dbSession.add_all(load_photometry)
db.dbSession.commit()

### Load photometry, such as it is, for object that are not in the tractor catalog.

In [91]:
from astropy.table import join
loaded_targetid = Table()
loaded_targetid['TARGETID'] = np.array([r.targetid for r in load_photometry])
loaded_targetid['LS_ID'] = np.array([r.ls_id for r in load_photometry])
j = join(potential_targetphot['TARGETID', 'RELEASE'], loaded_targetid, join_type='left', keys='TARGETID')
load_rows = np.zeros((len(potential_targetphot),), dtype=bool)
try:
    load_targetids = j['TARGETID'][j['LS_ID'].mask]
except AttributeError:
    #
    # This means *every* TARGETID is already loaded.
    #
    pass
else:
    unique_targetid, targetid_index = np.unique(potential_targetphot['TARGETID'].data, return_index=True)
    for t in load_targetids:
        load_rows[targetid_index[unique_targetid == t]] = True
load_rows

array([ True,  True,  True, ..., False, False, False])

In [92]:
row_index = load_rows
expand_dchisq = ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Photometry.__table__.columns:
        if c.name == 'ls_id':
            ls_id = ((potential_targetphot['RELEASE'].data.astype(np.int64) << 40) |
                     (potential_targetphot['BRICKID'].data.astype(np.int64) << 16) |
                     potential_targetphot['BRICK_OBJID'].data.astype(np.int64))
            data_columns.append(ls_id.tolist())
        elif c.name in expand_dchisq:
            j = expand_dchisq.index(c.name)
            data_columns.append(potential_targetphot[row_index]['DCHISQ'][:, j].tolist())
        else:
            data_columns.append(potential_targetphot[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_targetphot = [db.Photometry(**(dict([(c.name, d) for c, d in zip(db.Photometry.__table__.columns, row)]))) for row in data_rows]
load_targetphot[:20]

[Photometry(targetid=2404005925879810),
 Photometry(targetid=2404005925879811),
 Photometry(targetid=2404005925879812),
 Photometry(targetid=2404005925879813),
 Photometry(targetid=2404008190803968),
 Photometry(targetid=2404008190803969),
 Photometry(targetid=2404008199192577),
 Photometry(targetid=2404008199192578),
 Photometry(targetid=2404008207581184),
 Photometry(targetid=2404008207581185),
 Photometry(targetid=2404008207581187),
 Photometry(targetid=2404008207581188),
 Photometry(targetid=2404010489282561),
 Photometry(targetid=2404010489282562),
 Photometry(targetid=2404010497671169),
 Photometry(targetid=2404010497671170),
 Photometry(targetid=2404010501865477),
 Photometry(targetid=2404010501865478),
 Photometry(targetid=2404012741623809),
 Photometry(targetid=2404012741623810)]

In [94]:
# db.dbSession.rollback()
db.dbSession.add_all(load_targetphot)
db.dbSession.commit()

In [95]:
from specprodDB.util import surveyid
row_index = np.arange(len(potential_targetphot))
if len(row_index) > 0:
    data_columns = list()
    for c in db.Target.__table__.columns:
        if c.name == 'id':
            s = np.array([surveyid(s) for s in potential_targetphot['SURVEY']], dtype=np.int64)
            id0 = s << 32 | potential_targetphot['TILEID'].base.astype(np.int64)
            data_columns.append([i0 << 64 | i1 for i0, i1 in zip(id0.tolist(), potential_targetphot['TARGETID'].tolist())])
        else:
            data_columns.append(potential_targetphot[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_target = [db.Target(**(dict([(c.name, d) for c, d in zip(db.Target.__table__.columns, row)]))) for row in data_rows]
load_target[:20]

[Target(targetid=2404005925879810, tileid=3867, survey='main'),
 Target(targetid=2404005925879811, tileid=3867, survey='main'),
 Target(targetid=2404005925879812, tileid=3867, survey='main'),
 Target(targetid=2404005925879813, tileid=3867, survey='main'),
 Target(targetid=2404008190803968, tileid=3867, survey='main'),
 Target(targetid=2404008190803969, tileid=3867, survey='main'),
 Target(targetid=2404008199192577, tileid=3867, survey='main'),
 Target(targetid=2404008199192578, tileid=3867, survey='main'),
 Target(targetid=2404008207581184, tileid=3867, survey='main'),
 Target(targetid=2404008207581185, tileid=3867, survey='main'),
 Target(targetid=2404008207581187, tileid=3867, survey='main'),
 Target(targetid=2404008207581188, tileid=3867, survey='main'),
 Target(targetid=2404010489282561, tileid=3867, survey='main'),
 Target(targetid=2404010489282562, tileid=3867, survey='main'),
 Target(targetid=2404010497671169, tileid=3867, survey='main'),
 Target(targetid=2404010497671170, tilei

In [96]:
# db.dbSession.rollback()
db.dbSession.add_all(load_target)
db.dbSession.commit()

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [97]:
new_tile.lastnight

20211126

In [103]:
from glob import glob
redshift_dir = os.path.join(specprod_dir, 'tiles', 'cumulative', str(new_tile.tileid), str(new_tile.lastnight))
redrock_files = glob(os.path.join(redshift_dir, f'redrock-*-{new_tile.tileid:d}-thru{new_tile.lastnight:d}.fits'))

In [104]:
with fits.open(redrock_files[0]) as hdulist:
    hdulist.info()

Filename: /global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/3867/20211126/redrock-0-3867-thru20211126.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      25   ()      
  1  REDSHIFTS     1 BinTableHDU     31   500R x 11C   [K, D, 10D, D, D, K, K, 6A, 20A, K, D]   
  2  FIBERMAP      1 BinTableHDU    165   500R x 78C   [K, I, J, K, J, J, D, D, E, E, E, E, K, B, 3A, E, E, J, D, J, I, 8A, J, J, 4A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, E, E, K, 2A, E, E, E, E, 1A, K, K, K, K, K, K, D, D, J, I, E, I, I, E, E, E, E, D, E, D, E, E, E, E]   
  3  EXP_FIBERMAP    1 BinTableHDU     61   500R x 26C   [K, J, D, J, J, D, J, D, I, J, K, J, J, E, E, E, D, D, K, D, D, D, D, D, D, D]   
  4  TSNR2         1 BinTableHDU     75   500R x 33C   [K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E]   


In [105]:
from specprodDB.util import spgrpid

redshifts_table = Table.read(redrock_files[0], format='fits', hdu='REDSHIFTS')
row_index = no_sky(redshifts_table)
if len(row_index) > 0:
    data_columns = list()
    for c in db.Ztile.__table__.columns:
        if c.name == 'id':
            # ispgrpid('cumulative') << 27 | np.array([new_tile.tileid]*len(row_index), dtype=np.int64)
            # s = np.array([surveyid(s) for s in potential_targetphot['SURVEY']], dtype=np.int64)
            # id0 = s << 32 | potential_targetphot['TILEID'].base.astype(np.int64)
            data_columns.append([i0 << 64 | i1 for i0, i1 in zip(id0.tolist(), redshift_table[row_index]['TARGETID'].tolist())])
        elif c.name == 'targetphotid':
            pass
        elif c.name == 'desiname':
            pass
        elif c.name == 'survey':
            pass
        elif c.name == 'program':
            pass
        elif c.name.startswith('coeff_'):
            coeff_index = int(c.name.split('_')[1])
            data_columns.append(redshifts_table[row_index]['COEFF'][:, coeff_index].tolist())
        else:
            data_columns.append(redshifts_table[row_index][c.name.upper()].tolist())
data_rows = list(zip(*data_columns))
load_target = [db.Ztile(**(dict([(c.name, d) for c, d in zip(db.Ztile.__table__.columns, row)]))) for row in data_rows]


TARGETID,CHI2,COEFF,Z,ZERR,ZWARN,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2
int64,float64,float64[10],float64,float64,int64,int64,bytes6,bytes20,int64,float64
39633471919883859,7998.963254109025,110.12780043544497 .. 0.13773131303291555,0.9781954784122653,9.654987418244113e-05,0,7824,GALAXY,--,10,173.68104003369808
39633471919884932,8409.675214409828,53.05308690909152 .. 2.977051814055582,1.1112041245725723,4.217208912378726e-05,0,7857,GALAXY,--,10,275.05217918753624
39633471924077291,8212.116587935016,32.68379473938596 .. -10.521191299509509,0.18996502352739827,5.902911460011966e-05,4,7797,GALAXY,--,10,0.13148883846588433
39633474197391529,7920.530525684357,119.2708165816761 .. 5.607241781369166,0.7368729715313583,0.00012100970452797675,0,7830,GALAXY,--,10,118.78081225976348
616094224231695275,7415.548916250467,2.5077367070673215 .. 2.5269125883256707,1.239283759915375,0.00010321024007675009,5,7829,GALAXY,--,10,0.7009882144629955
39633471924076786,7360.269385820216,1949.5017952103508 .. 0.0,0.0003862733291083632,8.247428601686017e-06,0,7839,STAR,F,5,3703.1709775869194
39633471924078031,9149.331131741405,0.0007960051199296652 .. 0.0,2.10165104027563,0.0001953010051423887,0,7810,QSO,--,4,3824.1591387689114
39633474197390347,7744.641185134649,-6.4672971699429604 .. -0.4016652085526882,1.101643882919417,6.805660806674079e-05,0,7845,GALAXY,--,10,46.19608053565025
39633471919884249,9923.119987785816,0.00013930826554829355 .. 0.0,3.78307363140595,0.0002728128909010603,4,7828,QSO,--,4,4.681105062365532


In [106]:
[c.name for c in db.Ztile.__table__.columns]

['id',
 'targetphotid',
 'targetid',
 'desiname',
 'survey',
 'program',
 'spgrp',
 'spgrpval',
 'z',
 'zerr',
 'zwarn',
 'chi2',
 'coeff_0',
 'coeff_1',
 'coeff_2',
 'coeff_3',
 'coeff_4',
 'coeff_5',
 'coeff_6',
 'coeff_7',
 'coeff_8',
 'coeff_9',
 'npixels',
 'spectype',
 'subtype',
 'ncoeff',
 'deltachi2',
 'coadd_fiberstatus',
 'tileid',
 'coadd_numexp',
 'coadd_exptime',
 'coadd_numnight',
 'coadd_numtile',
 'mean_delta_x',
 'rms_delta_x',
 'mean_delta_y',
 'rms_delta_y',
 'mean_fiber_ra',
 'std_fiber_ra',
 'mean_fiber_dec',
 'std_fiber_dec',
 'mean_psf_to_fiber_specflux',
 'mean_fiber_x',
 'mean_fiber_y',
 'tsnr2_gpbdark_b',
 'tsnr2_elg_b',
 'tsnr2_gpbbright_b',
 'tsnr2_lya_b',
 'tsnr2_bgs_b',
 'tsnr2_gpbbackup_b',
 'tsnr2_qso_b',
 'tsnr2_lrg_b',
 'tsnr2_gpbdark_r',
 'tsnr2_elg_r',
 'tsnr2_gpbbright_r',
 'tsnr2_lya_r',
 'tsnr2_bgs_r',
 'tsnr2_gpbbackup_r',
 'tsnr2_qso_r',
 'tsnr2_lrg_r',
 'tsnr2_gpbdark_z',
 'tsnr2_elg_z',
 'tsnr2_gpbbright_z',
 'tsnr2_lya_z',
 'tsnr2_bgs_z',
 '